In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
df.shape

(28, 3)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

In [5]:
len(X_train)

22

In [6]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [7]:
def sigmoid_numpy(x):
    return 1/(1+np.exp(-x))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [8]:
# Define a value which is close to 0, add it to the '0' log number so as it may not become zero

eplison = 1e-15

In [9]:
# Define calculation above into a function

def log_loss(y_true, y_predicted):
    y_predicted_new = [max(i,eplison) for i in y_predicted]
    y_predicted_new = [min(i,1-eplison) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [10]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    def fit(self, X, y, epochs, loss_threshold):
        self.w1, self.w2, self.bias = self.gradient_decent(X['age'], X['affordibility'], y, epochs, loss_threshold)

    def predict(self, X_test):
        weighted_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.bias
        return sigmoid_numpy(weighted_sum)
    
    def gradient_decent(self, age, affordibility, y_true, epochs, loss_threshold):
        # w1, w2, bias
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)

        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordibility + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            
            loss = log_loss(y_true, y_predicted)

            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted - y_true))
            w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted - y_true))

            bias_d = np.mean(y_predicted-y_true)

            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d

            if i%50==0:
                print(f'Epoch: {i}, w1: {w1}, w1: {w2}, bias: {bias}, loss: {loss}')

            if loss<=loss_threshold:
                print(f'Epoch: {i}, w1: {w1}, w1: {w2}, bias: {bias}, loss: {loss}')
                break

        return w1, w2, bias

In [11]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs=8000, loss_threshold=0.4631)

Epoch: 0, w1: 0.974907633470177, w1: 0.948348125394529, bias: -0.11341867736368583, loss: 0.7113403233723417
Epoch: 50, w1: 1.5033195541731388, w1: 1.108384790367645, bias: -1.2319047301235464, loss: 0.5675865113475955
Epoch: 100, w1: 2.200713131760032, w1: 1.2941584023238903, bias: -1.6607009122062801, loss: 0.5390680417774752
Epoch: 150, w1: 2.8495727769689085, w1: 1.3696895491572747, bias: -1.986105845859897, loss: 0.5176462164249294
Epoch: 200, w1: 3.4430169708818035, w1: 1.404221862446503, bias: -2.2571369883752723, loss: 0.5005011269691375
Epoch: 250, w1: 3.9824504946495765, w1: 1.4239127329321228, bias: -2.4943773659718005, loss: 0.48654089537617085
Epoch: 300, w1: 4.472179522095915, w1: 1.438787986553552, bias: -2.707387811922373, loss: 0.4750814640632793
Epoch: 350, w1: 4.917245868007634, w1: 1.4525660781176122, bias: -2.901176333556766, loss: 0.46561475306999006
Epoch: 366, w1: 5.051047623653049, w1: 1.4569794548473887, bias: -2.9596534546250037, loss: 0.46293944095888917


In [12]:
customModel.predict(X_test_scaled)

2     0.705020
10    0.355836
21    0.161599
11    0.477919
14    0.725586
9     0.828987
dtype: float64